In [54]:
from PIL import Image, ImageDraw, ImageOps
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
import scipy as sc

## Gif processing

In [55]:
def process_gif(file:str)->np.array:
    """
    Return the input file (gif) as a numpy array
    """
    # Open the original GIF file
    with Image.open(file) as im:
        w, h = im.size 
        n = im.n_frames

        # Create an empty numpy array to hold the frames of the new GIF file
        out = np.empty((n, h, w, 3), dtype=np.uint8)

        # Loop over each frame of the original GIF file and copy it into the numpy array
        for i in range(n):
            im.seek(i)
            out[i] = np.array(im.convert('RGB'))
    return out

In [56]:
out = process_gif("catgif.gif")
# Convert the numpy array back to a list of PIL Image objects
frames = [Image.fromarray(out[i], mode='RGB') for i in range(len(out))]

# Save the new GIF file
frames[0].save('new_catgif.gif', format='GIF', save_all=True, append_images=frames[1:], duration=80, loop=0)

## Some filters

In [57]:
#FLIP THE CAT
out_flipped = np.flip(out, axis=2)

frames = [Image.fromarray(out_flipped[i], mode='RGB') for i in range(len(out_flipped))]

frames[0].save('flip_catgif.gif', format='GIF', save_all=True, append_images=frames[1:], duration=80, loop=0)

In [58]:
#DRUNK CAT
out_flipped = sc.ndimage.filters.gaussian_filter(out,1.5)


frames = [Image.fromarray(out_flipped[i], mode='RGB') for i in range(len(out_flipped))]

frames[0].save('drunk_catgif.gif', format='GIF', save_all=True, append_images=frames[1:], duration=80, loop=0)

C:\Users\jagiraldo\AppData\Local\Temp\ipykernel_1344\3359974845.py:2: DeprecationWarning: Please use `gaussian_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  out_flipped = sc.ndimage.filters.gaussian_filter(out,1.5)


In [59]:
# ANTI-CAT
out_flipped = np.invert(out)

# Convert the modified numpy array back to a list of PIL Image objects
frames = [Image.fromarray(out_flipped[i], mode='RGB') for i in range(len(out_flipped))]

# Save the modified GIF file
frames[0].save('anti_catgif.gif', format='GIF', save_all=True, append_images=frames[1:], duration=80, loop=0)

In [60]:
out_rgb = out[:, :, :, :3]
filter_value = -10

b_filter = np.clip(out_rgb - filter_value, 0, 255)

frames = [Image.fromarray(b_filter[i], mode='RGB') for i in range(len(b_filter))]

frames[0].save('this_catgif.gif', format='GIF', save_all=True, append_images=frames[1:], duration=80, loop=0)

In [61]:
out_rgb = out[:, :, :, :3]
r_filter_value = 20
g_filter_value = 18
b_filter_value = 9

# Subtract the filter value from each RGB channel
r_filter = np.clip(out_rgb[:, :, :, 0] - r_filter_value, 0, 255)
g_filter = np.clip(out_rgb[:, :, :, 1] - g_filter_value, 0, 255)
b_filter = np.clip(out_rgb[:, :, :, 2] - b_filter_value, 0, 255)

# Combine the filtered RGB channels into a single numpy array
b_filter = np.stack([r_filter, g_filter, b_filter], axis=-1)

# Create a list of PIL Image objects from the modified numpy array
frames = [Image.fromarray(b_filter[i], mode='RGB') for i in range(len(b_filter))]

# Save the new GIF file
frames[0].save('3new_catgif.gif', format='GIF', save_all=True, append_images=frames[1:], duration=80, loop=0)

## Add something to the video

In [62]:
# Create a red heart-shaped figure to paste onto each frame

line_size = (300,40)
line_color = (0,0,0,0)
line = Image.new('RGBA',line_size,line_color)
draw = ImageDraw.Draw(line)

Loop over each frame of the original GIF file and copy it into the array

In [63]:
im = Image.open('catgif.gif')
frames=[]

for i in range (len(out)):
    im.seek(i)
    points = [(10+(i*5),10),(40+5*i,40)]
    draw.ellipse(points,fill = "blue" )
    frame = np.array(im.convert('RGBA'))
    #paste it onto the frame
    frame[0:line_size[1],0:line_size[0]] = np.array(line)
    frames.append(Image.fromarray(frame,mode='RGBA'))
    
    
#save the new gif 
frames[0].save("new_catgif_with_line.gif", format='GIF', save_all=True, append_images=frames[1:],duration=80,loop=0)

## Frame recognition

We calculate for each its frame mean pixel value and compare it with the value of a black pixel

In [64]:
prom = (np.apply_over_axes(np.sum, out, [1,2,3])/(382*300)).reshape(-1)
ref_value = 0 # ref value for black

comparison = np.abs(prom-ref_value)

detection = comparison>450 # This was selected by seeing the array values, there is no good prediction using the image mean pixel value

detection = pd.Series(detection) # Use series to filter using indexes
detection = detection[detection]

d = out[detection.index]

Now we store the detected frames as Image objects

In [65]:
frames_detect = [Image.fromarray(d[i], mode='RGB') for i in range(len(d))] # revert detected frames 
print(len(frames_detect))
frames_detect[15].show()


24


Note that using the average pixel value of the image was not a good indicator for detecting the black piece in the image